In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer 

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from os import listdir

In [2]:
'''from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)'''

"from sklearn.datasets import fetch_20newsgroups\nnewsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)\nnewsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)"

In [3]:
'''print(list(newsgroups_train.target_names))'''


'print(list(newsgroups_train.target_names))'

In [2]:
list_doc=[]
doc = ""
for i,x in enumerate(['business','entertainment','politics','sport','tech']):
    for y in listdir('bbc/'+x+'/'):
        list_doc.append(open('bbc/'+x+'/'+y,"r").read())

In [3]:
from gensim.parsing.preprocessing import STOPWORDS


In [4]:
def preprocess(text):
    return [w for w in gensim.utils.simple_preprocess(text) if w not in STOPWORDS and len(w)>3]
def lemmatize(text):
    return [WordNetLemmatizer().lemmatize(w) for w in text]

In [55]:
preproc_doc = []
for s in list_doc:
    preproc_doc.append(lemmatize(preprocess(s)))
    

In [6]:
preproc_doc[:2]

[['sale',
  'boost',
  'time',
  'warner',
  'profit',
  'quarterly',
  'profit',
  'medium',
  'giant',
  'timewarner',
  'jumped',
  'month',
  'december',
  'year',
  'earlier',
  'firm',
  'biggest',
  'investor',
  'google',
  'benefited',
  'sale',
  'high',
  'speed',
  'internet',
  'connection',
  'higher',
  'advert',
  'sale',
  'timewarner',
  'said',
  'fourth',
  'quarter',
  'sale',
  'rose',
  'profit',
  'buoyed',
  'gain',
  'offset',
  'profit',
  'warner',
  'bros',
  'user',
  'time',
  'warner',
  'said',
  'friday',
  'owns',
  'search',
  'engine',
  'google',
  'internet',
  'business',
  'mixed',
  'fortune',
  'lost',
  'subscriber',
  'fourth',
  'quarter',
  'profit',
  'lower',
  'preceding',
  'quarter',
  'company',
  'said',
  'underlying',
  'profit',
  'exceptional',
  'item',
  'rose',
  'stronger',
  'internet',
  'advertising',
  'revenue',
  'hope',
  'increase',
  'subscriber',
  'offering',
  'online',
  'service',
  'free',
  'timewarner',
  'i

In [56]:
dwords = gensim.corpora.Dictionary(preproc_doc)
len(dwords.token2id)

23586

In [57]:
dwords.filter_extremes(no_below=7, no_above=0.5, keep_n=10000)

In [58]:
c = 0
for k,v in dwords.iteritems():
    c+=1
    print(k,v)
    if c==10:
        break

0 account
1 advert
2 advertising
3 alexander
4 analyst
5 aside
6 better
7 biggest
8 book
9 boost


In [59]:
bow = [dwords.doc2bow(s) for s in preproc_doc]

In [60]:
bow

[[(0, 2),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 2),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 3),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 2),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 4),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 2),
  (89, 1),
  (90, 1),
  (91, 1)

In [61]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.09035156145167317),
 (1, 0.07201525719460188),
 (2, 0.13862183122905988),
 (3, 0.07525300490395335),
 (4, 0.03514686928575605),
 (5, 0.06881955158385887),
 (6, 0.03441249591633964),
 (7, 0.03585009934423272),
 (8, 0.04833863308683729),
 (9, 0.04674453602419089),
 (10, 0.06981740088894371),
 (11, 0.056469917168980845),
 (12, 0.09263507501934999),
 (13, 0.03244063134383442),
 (14, 0.040070794164209445),
 (15, 0.041206380445927074),
 (16, 0.029842823939285067),
 (17, 0.042041124125492046),
 (18, 0.04934520264254004),
 (19, 0.046917576949950524),
 (20, 0.05579800866639994),
 (21, 0.0642059290529028),
 (22, 0.09695742910142462),
 (23, 0.06967377777526082),
 (24, 0.03823563696111441),
 (25, 0.05919877025418778),
 (26, 0.0702937385715121),
 (27, 0.059747470152040426),
 (28, 0.043122433449017956),
 (29, 0.06573814832591925),
 (30, 0.08342409425980502),
 (31, 0.058931130655521864),
 (32, 0.07143996851658484),
 (33, 0.08896224983806883),
 (34, 0.05106291346117824),
 (35, 0.035146869285756

In [62]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dwords, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.003*"award" + 0.002*"best" + 0.002*"film" + 0.002*"music" + 0.002*"virus" + 0.002*"band" + 0.001*"oscar" + 0.001*"user" + 0.001*"album" + 0.001*"number"
Topic: 1 Word: 0.003*"game" + 0.002*"election" + 0.002*"labour" + 0.002*"blair" + 0.002*"party" + 0.002*"england" + 0.002*"tory" + 0.002*"sale" + 0.001*"government" + 0.001*"injury"
Topic: 2 Word: 0.002*"mobile" + 0.002*"market" + 0.002*"china" + 0.002*"phone" + 0.002*"bank" + 0.002*"firm" + 0.002*"music" + 0.002*"company" + 0.002*"film" + 0.002*"growth"
Topic: 3 Word: 0.002*"game" + 0.002*"yukos" + 0.002*"player" + 0.001*"company" + 0.001*"film" + 0.001*"people" + 0.001*"club" + 0.001*"united" + 0.001*"mail" + 0.001*"league"
Topic: 4 Word: 0.002*"film" + 0.002*"election" + 0.002*"labour" + 0.002*"brown" + 0.002*"lord" + 0.002*"tory" + 0.002*"chancellor" + 0.001*"blair" + 0.001*"award" + 0.001*"government"


In [71]:
bow_test = dwords.doc2bow(lemmatize(preprocess(list_doc[1])))

for index, score in sorted(lda_model_tfidf[bow_test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

Score: 0.9953972697257996	 Topic: 0.002*"mobile" + 0.002*"market" + 0.002*"china" + 0.002*"phone" + 0.002*"bank" + 0.002*"firm" + 0.002*"music" + 0.002*"company" + 0.002*"film" + 0.002*"growth"


In [64]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.9954,"mobile, market, china, phone, bank, firm, musi...","[sale, boost, time, warner, profit, quarterly,..."
1,1,2.0,0.9954,"mobile, market, china, phone, bank, firm, musi...","[dollar, gain, greenspan, speech, dollar, high..."
2,2,3.0,0.8652,"game, yukos, player, company, film, people, cl...","[yukos, unit, buyer, face, loan, claim, owner,..."
3,3,2.0,0.9951,"mobile, market, china, phone, bank, firm, musi...","[high, fuel, price, profit, british, airway, b..."
4,4,2.0,0.9912,"mobile, market, china, phone, bank, firm, musi...","[pernod, takeover, talk, lift, domecq, share, ..."
5,5,2.0,0.9908,"mobile, market, china, phone, bank, firm, musi...","[japan, narrowly, escape, recession, japan, ec..."
6,6,2.0,0.9931,"mobile, market, china, phone, bank, firm, musi...","[job, growth, slow, created, fewer, job, expec..."
7,7,2.0,0.9944,"mobile, market, china, phone, bank, firm, musi...","[india, call, fair, trade, rule, india, attend..."
8,8,2.0,0.9926,"mobile, market, china, phone, bank, firm, musi...","[ethiopia, crop, production, ethiopia, produce..."
9,9,4.0,0.5347,"film, election, labour, brown, lord, tory, cha...","[court, reject, tobacco, case, government, cla..."


In [65]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, bow, dictionary=lda_model_tfidf.id2word)
vis

C:\Users\jinit\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.030893  0.016356       1        1  41.136284
1      0.007862 -0.024080       2        1  22.921410
3     -0.001530 -0.004853       3        1  18.475883
0     -0.022189  0.018771       4        1  10.867340
4     -0.015035 -0.006193       5        1   6.599079, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
1910  Default  306.000000       award  306.000000  30.0000  30.0000
503   Default  373.000000    election  373.000000  29.0000  29.0000
39    Default  578.000000        film  578.000000  28.0000  28.0000
1255  Default  364.000000      labour  364.000000  27.0000  27.0000
682   Default  334.000000        best  334.000000  26.0000  26.0000
542   Default  292.000000       brown  292.000000  25.0000  25.0000
68    Default  204.000000        lord  204.000000  24.0000  24.0000
5455  Default  266.000000        tory  266.000000  23.0000  23.0000
1239  Default  161.000000  chancellor  161.000000  22.0000  22.0000
2212  Default  132.000000       virus  132.000000  21.0000  21.0000
4659  Default  174.000000       oscar  174.000000  20.0000  20.0000
1095  Default  339.000000       party  339.000000  19.0000  19.0000
3494  Default  184.000000       album  184.000000  18.0000  18.0000
2714  Default  359.000000       blair  359.000000  17.0000  17.0000
2476  Default  234.000000      howard  234.000000  16.0000  16.0000
315   Default  165.000000       yukos  165.000000  15.0000  15.0000
4975  Default  190.000000        band  190.000000  14.0000  14.0000
2697  Default  165.000000      school  165.000000  13.0000  13.0000
4472  Default   89.000000       davis   89.000000  12.0000  12.0000
4625  Default  188.000000       chart  188.000000  11.0000  11.0000
170   Default  399.000000  government  399.000000  10.0000  10.0000
4254  Default  171.000000       actor  171.000000   9.0000   9.0000
5888  Default   83.000000     roddick   83.000000   8.0000   8.0000
78    Default  407.000000       music  407.000000   7.0000   7.0000
4718  Default  146.000000    festival  146.000000   6.0000   6.0000
842   Default  178.000000        mail  178.000000   5.0000   5.0000
5572  Default   76.000000        ukip   76.000000   4.0000   4.0000
1554  Default  204.000000     ireland  204.000000   3.0000   3.0000
4366  Default  113.000000  nomination  113.000000   2.0000   2.0000
3893  Default  159.000000        song  159.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
1239   Topic5   34.867374  chancellor  161.529907   1.1851  -6.4774
1665   Topic5   13.987356      arrest   56.837524   1.3162  -7.3908
5496   Topic5   18.914015    blunkett   83.491295   1.2334  -7.0890
3169   Topic5   16.039751        duty   70.275421   1.2409  -7.2539
542    Topic5   44.607925       brown  292.094818   0.8391  -6.2310
503    Topic5   51.734089    election  373.133392   0.7424  -6.0828
2697   Topic5   28.448109      school  165.528061   0.9572  -6.6809
625    Topic5   22.496885        seed  119.313995   1.0499  -6.9156
1255   Topic5   48.335041      labour  364.874939   0.6968  -6.1508
4718   Topic5   24.979862    festival  146.341507   0.9504  -6.8109
5455   Topic5   35.432247        tory  266.070465   0.7021  -6.4613
39     Topic5   53.944328        film  578.036133   0.3466  -6.0410
870    Topic5   22.994444   secretary  147.048660   0.8627  -6.8937
177    Topic5   24.912449       house  176.127808   0.7624  -6.8136
2714   Topic5   33.871605       blair  359.182739   0.3570  -6.5064
640    Topic5   24.331861    campaign  184.693466   0.6913  -6.8372
1910   Topic5   29.907122       award  306.376007   0.3915  -6.6308
1095   Topic5   29.612490       party  339.139374   0.2800  -6.6407
170    Topic5   29.625130  government  399.372192   0.1170  -6.6403
682    Topic5   27.588999        best  334.

In [ ]:
def pipeline(doc):
    bow_doc = dwords.doc2bow(lemmatize(preprocess(doc)))
    for index, score in sorted(lda_model_tfidf[bow_test], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']    
    return [i for i in df_dominant_topic['Keywords'] if i in df_dominant_topic['Text']]